# Sample queries for GEMD database v1: only using GEMDContext tables
These queries only access the GEMDContext table. \
Apart from gemd_type and uid, all information needs to be extracted from the JSON context.

In [ ]:
import SciServer.CasJobs as cj
from pprint import pprint
import json
import pandas

In [ ]:
# pandas.set_option()
pandas.options.display.max_rows=None

In [ ]:
DATABASE='GEMD'

## count both contexts and objects

In [ ]:
sql="""
select 'object',count(*) as num from gemdobject
union
select 'context',count(*) from gemdcontext
"""
cj.executeQuery(sql, DATABASE)

## count occurrences of gemd_types in GEMDContext

In [ ]:
sql="""
select distinct gemd_type
,      x.[key]
  from gemdcontext cross apply openjson(context,'$') x
  where x.[key] not in ('tags','description','uids','name','type','bounds','file_links','labels')
 order by 1,2
"""
cj.executeQuery(sql, DATABASE)

## count occurrences of gemd_types for GEMDContext with a GEMDObject

In [ ]:
sql="""
select c.gemd_type, count(*) as num
  from gemdcontext c join gemdobject o on o.uid=c.uid
 group by c.gemd_type
 order by 1
"""
cj.executeQuery(sql, DATABASE)

## count occurrences of GEMDContext without a GEMDObject
Ingredient run and process run are missing gemd_types missing from GEMDObject. \
But there are measurement_run and material_run entries of GEMDContext both with and without a matching GEMDObject.

In [ ]:
sql="""
select c.gemd_type, count(*) as num
  from gemdcontext c 
  where not exists (select gemd_type from gemdobject o where o.uid=c.uid)
 group by c.gemd_type
 order by 1
"""
cj.executeQuery(sql, DATABASE)

## find some "virgin" measurement_run and process_run entries in GEMDContext
seems there are measurement_run and process_run entries in GEMDContext with and without GEMDObject

In [ ]:
sql="""
select top 10 c.*
  from gemdcontext c 
  where c.gemd_type in ('measurement_run','process_run')
    and not exists (select gemd_type from gemdobject o where o.uid=c.uid)
 order by newid()     -- random ordering
"""
cj.executeQuery(sql, DATABASE)

# JSON queries
Exploring JSON query support in MS SQL.
See [MS SQL docs](https://docs.microsoft.com/en-us/sql/relational-databases/json/json-data-sql-server?view=sql-server-ver15) for information.
Following queries are pursued
* find all runs for some random spec. Possibly specific types suchs as material_run and material_spec
* for a material_spec having non-empty properties, get material_run(s) and material_template
* find properties defined on templates
* find all properties for a material_run

In [ ]:
sql="""
select top 4 context
  from  gemdcontext c where gemd_type='process_run'
 order by newid()
"""
process_runs=cj.executeQuery(sql,DATABASE)
for process_run in process_runs.itertuples():
    pprint(process_run.context)

## find all the JSON keys at level one in the context field for different gemd_type values

In [ ]:

sql="""
SELECT distinct m.gemd_type,x.[Key]--,x.Value
FROM gemdcontext m
cross apply OPENJSON(m.context, '$') AS x
where m.gemd_type like 'process%'
order by 1,2
"""
cj.executeQuery(sql, DATABASE)


## find process runs for a random set of process specs

In [ ]:
sql="""
with some_specs as (  
select top 10 *              -- using SELECT TOP N ... ORDER BY newid() is standard trick for getting a random sample of N entries
  from gemdcontext
 where gemd_type='process_spec' 
order by newid()                       
)
select s.uid as spec_id
,      json_value(s.context,'$.name') as spec
,      r.uid as run_id
,      r.context as run
  from some_specs s
  join gemdcontext r
 on r.gemd_type='process_run' 
   and JSON_VALUE(r.context,'$.spec.id')=s.uid
order by spec
"""
cj.executeQuery(sql,DATABASE)

## for a material_spec having non-empty properties, get material_run(s) and material_template

In [ ]:
%%time
sql="""
with some_specs as (
select top 3 *
  from gemdcontext
 where gemd_type='material_spec' 
   and context like '%"properties":%'
and context not like '%properties": [[]]%'   -- having some non-trivial properties. NOTE '[' must be escaped to '[[]', ']' needs no escaping
                                             -- see https://docs.microsoft.com/en-us/sql/t-sql/language-elements/like-transact-sql?view=sql-server-ver16#using-wildcard-characters-as-literals
)
select s.uid as spec_id
,      JSON_VALUE(t.context,'$.name') as template
,      JSON_VALUE(s.context,'$.name') as spec
,      JSON_VALUE(m.context,'$.name') as material
,      JSON_VALUE(p.context,'$.name') as process
,      JSON_VALUE(ps.context,'$.name') as process_spec
--,      t.context as t_context
--,      s.context as s_context
--,      m.context as m_context
,      json_query(ps.context,'$.tags') as ps_tags
--,      p.context as p_context
--,      ps.context as ps_context
  from some_specs s
  join gemdcontext t on t.uid=json_value(s.context,'$.template.id') -- get template for material_spec
  join gemdcontext m on s.uid=json_value(m.context,'$.spec.id')     -- get material_run for material_spec
  join gemdcontext p on p.uid=json_value(m.context,'$.process.id')  -- get process_run for material_run
  join gemdcontext ps on ps.uid=json_value(p.context,'$.spec.id')   -- get process_spec for process_run
  """
df=cj.executeQuery(sql,DATABASE)
df.head(4)

In [ ]:
specs=df.groupby(['spec_id','spec'])
specs['material'].count()

In [ ]:
for k,group in specs:
    spec=specs.get_group(k)
    break
spec

In [ ]:
for t in spec.itertuples():
    t_context=json.loads(t.t_context)
    s_context=json.loads(t.s_context)
    m_context=json.loads(t.m_context)
    p_context=json.loads(t.p_context)
    ps_context=json.loads(t.ps_context)
    print('TEMPLATE:')
    pprint.pprint(t_context)
    print('SPEC.properties:')
    pprint.pprint(s_context['properties'])
    pprint.pprint(m_context)
    pprint.pprint(ps_context)
    print('=======================')
    break

## for a material_spec get properties

In [ ]:
%%time
sql="""
select s.uid as spec_id
,      JSON_VALUE(s.context,'$.name') as spec
,      JSON_VALUE(sp.property,'$.name') as p
,      sp.property
  from GEMDContext s
  cross apply OPENJSON(s.context,'$.properties') with (property nvarchar(max) 'lax$.property' as json) sp
 where s.gemd_type='material_spec' 
  """
df=cj.executeQuery(sql,DATABASE)
# df.head(4)
df

## find properties defined on templates

In [ ]:
%%time
sql="""
select t.uid as template_uid
,      JSON_VALUE(t.context,'$.name') as template
,      JSON_VALUE(t.context,'$.type') as template_type
,      p2.uid as prop_uid
,      json_value(p.context,'$.name') as prop_name
,      json_value(p.context,'$.description') as prop_description
,      json_value(p.context,'$.bounds.type') as prop_bounds_type
,      json_value(p.context,'$.bounds.lower_bound') as prop_lower_bound
,      json_value(p.context,'$.bounds.upper_bound') as prop_upper_bound
,      json_value(p.context,'$.bounds.default_units') as prop_default_units
,      json_query(p.context,'$.bounds.categories') as prop_categories
,      p.context 
  from GEMDContext t 
  cross apply openjson(t.context,'$.properties')  with (props nvarchar(max) '$' as json ) p1  -- for templates, properties seem to be [[  ]], array-of-arrays
  cross apply openjson(p1.props, '$') with(uid varchar(36) '$.id') p2                          -- hence need double OPENJSON
  join gemdcontext p on p.uid=p2.uid
 where t.gemd_type like '%[_]template'
   and p2.uid is not null
   order by template_type,template,prop_name
"""
props=cj.executeQuery(sql,DATABASE)
props.head(3)

## find properties + values defined on *_spec

In [ ]:
sql="""
select spec.uid as spec_uid
,      JSON_VALUE(spec.context,'$.name') as spec_name
,      JSON_VALUE(spec.context,'$.type') as spec_type
,      pt.uid as prop_template_uid
,      json_value(pt.context,'$.name') as prop_template_name
,      json_value(pt.context,'$.type') as prop_template_type
,      json_value(pt.context,'$.description') as prop_template_description
,      json_value(pt.context,'$.bounds.type') as prop_template_bounds_type
,      json_value(pt.context,'$.bounds.lower_bound') as prop_template_lower_bound
,      json_value(pt.context,'$.bounds.upper_bound') as prop_template_upper_bound
,      json_value(pt.context,'$.bounds.default_units') as prop_template_default_units
,      json_query(pt.context,'$.bounds.categories') as prop_template_categories
,      json_query(pt.context,'$.bounds.components') as prop_template_components
,      json_value(p.property,'$.value.nominal') as prop_nominal_value
,      json_value(p.property,'$.value.units') as prop_units
,      json_value(p.property,'$.value.category') as prop_category
,      json_query(p.property,'$.value.quantities') as prop_quantities
,      p.property
  from GEMDContext spec
  cross apply openjson(spec.context,'$.properties')  
         with (property nvarchar(max) '$.property' as json ) p  -- for specs, properties seem to be [  ], simple arrays
  join gemdcontext pt on pt.uid=json_value(p.property, '$.template.id')
 where spec.gemd_type like '%[_]spec'
   and pt.uid is not null
   order by spec_type,spec_name,spec_uid,prop_template_name
"""
df=cj.executeQuery(sql,DATABASE)
df.head(10)

In [ ]:
for tags in df['spec_tags']:
    pprint(tags)

## find material_run data for material_spec-s

In [ ]:

sql="""
select spec.uid as spec_uid
,      JSON_VALUE(spec.context,'$.name') as spec_name
,      JSON_VALUE(run.context,'$.name') as run_name
,      run.context as run
  from GEMDContext spec
  join GEMDContext run on spec.uid=json_value(run.context,'$.spec.id')  
 where spec.gemd_type = 'material_spec'
   order by spec_name,spec_uid
"""
df=cj.executeQuery(sql,DATABASE)
df.head(3)


## find  file_links

In [ ]:
sql="""
select gemd_type, count(*) as num
  from gemdcontext
 where context like '%"file[_]links":%'
and context not like '%file[_]links": [[]]%'   -- having some non-trivial file_links. NOTE '[' must be escaped to '[[]', ']' needs no escaping
group by gemd_type
order by 1
"""
file_links=cj.executeQuery(sql,DATABASE)
file_links

In [ ]:
sql="""
select c.uid, c.gemd_type
,      json_query(max(c.context), '$.file_links') as filelinks
,      string_agg(fl.filename,' | ') as filenames
--, fl.url
  from gemdcontext c
  cross apply OPENJSON(c.context,'$.file_links') 
         with(filename varchar(100) '$.filename', url varchar(1024) '$.url') fl
 where fl.filename is not NULL and fl.filename != ''
 group by c.uid,c.gemd_type
order by 2,1
"""
file_links=cj.executeQuery(sql,DATABASE)
file_links

In [ ]:
for 

## find gemd_types with non-trival notes

In [ ]:
sql="""
select gemd_type, count(*) as num
  from gemdcontext
 where context like '%"notes":%'
and context not like '%notes": [[]]%'   -- having some non-trivial notes. NOTE '[' must be escaped to '[[]', ']' needs no escaping
group by gemd_type
order by 1
"""
notes=cj.executeQuery(sql,DATABASE)
notes

## find gemd_types with non-trival parameters
(Apparently there are none with ingredients.)

In [ ]:
sql="""
with a as (
select gemd_type
,      json_query(context,'$.parameters' ) as params
  from gemdcontext
)
select gemd_type, count(*) as num from a
where params is not null and params != '[]'
group by gemd_type
order by 1
"""
parameters=cj.executeQuery(sql,DATABASE)
parameters

In [ ]:
sql="""
with a as (
select top 3 *,json_query(context,'$.parameters' ) as params
  from gemdcontext c
 where c.gemd_type='process_spec'
   and json_query(context,'$.parameters' ) is not null and json_query(context,'$.parameters' ) != '[]'
order by newid()
)
select a.uid,json_value(a.context, '$.name') as spec_name
,      p.name, p.nominal_value, p.category_value
,      a.params
  from a
  cross apply openjson(a.params, '$') 
    with (name varchar(32) '$.name', nominal_value float '$.value.nominal', category_value varchar(32) '$.value.category') p
where params is not null and params != '[]'
order by 1,3
"""
parameters=cj.executeQuery(sql,DATABASE)
parameters

In [ ]:
pprint(parameters.iloc[0].params)

## find processes for material and who performed it

In [ ]:
sql="""
with a as (
select json_value(m.context,'$.name') as m_run
,      m.context as m
,      json_value(p.context,'$.name') as p_name
,      json_value(p.context,'$.type') as p_type
,      json_value(p.context,'$.source.performed_by') as p_performed_by
,      json_value(p.context,'$.source.performed_date') as p_performed_date
,      p.context as p
  from gemdcontext m
  left outer join gemdcontext p on p.uid=json_value(m.context,'$.process.id')
 where m.gemd_type='material_run'
)
select p_performed_by, count(*) as num
  from a
  where p_performed_by is not NULL and p_performed_by != ''
  group by p_performed_by
"""
cj.executeQuery(sql, DATABASE)

In [ ]:
sql="""
select c.gemd_type,count(*) as num
  from gemdcontext c
  where context like '%tags%'
  group by gemd_type
"""
cj.executeQuery(sql, DATABASE)